# Statistics

## Overview

We provide a number of built in procedures for computing sample statistics and running various common estimation procedures (e.g., variance-component estimation, association studies, and the like).  During each generation of a simulation, each procedure provided as a `statistics` attribute of a `sim.Simulation` object will be run. Of course, you can also configure `xftsim` to write genotypes and phenotypes to disk each generation and can thereby use any external software you like, as is detailed in [the post-processing section](./proc.py).

As a starting point, we'll initialize a small simulation that incorporates three `Statistic` objects: 

In [1]:
import xftsim as xft
from xftsim import stats
xft.config.print_durations_threshold = 10
n=2000; m=800

founder_haplotypes = xft.founders.founder_haplotypes_uniform_AFs(n=n, m=m)
architecture = xft.arch.GCTA_Architecture(h2=[.5,.4], phenotype_name=['height', 'BMD'], 
                                          haplotypes=founder_haplotypes)
recombination_map = xft.reproduce.RecombinationMap.constant_map_from_haplotypes(founder_haplotypes, 
                                                                                p =.1)
mating_regime = xft.mate.RandomMatingRegime(mates_per_female=1,
                                            offspring_per_pair=2)

sim = xft.sim.Simulation(founder_haplotypes=founder_haplotypes,
                         architecture=architecture,
                         recombination_map=recombination_map,
                         mating_regime=mating_regime,
                         statistics = [])

We didn't specify any statistics, so the simulation will just proceed through phenotype construction and mating each generation without saving any additional results.

## Accessing results

After adding a `Statistic`, we'll begin to accumulate results in the `results_store` attribute as we iterate. The `results_store` is just a dict of dicts that we can print nicely using `xft.utils.print_tree()`:

In [2]:
sim.statistics = [stats.SampleStatistics(means=False, 
                                         variances=False, 
                                         variance_components = False)]
sim.run(2)
xft.utils.print_tree(sim.results_store)

0: 
|__sample_statistics: 
|____vcov: <class 'pandas.core.frame.DataFrame'>
|____corr: <class 'pandas.core.frame.DataFrame'>
1: 
|__sample_statistics: 
|____vcov: <class 'pandas.core.frame.DataFrame'>
|____corr: <class 'pandas.core.frame.DataFrame'>


The `results_store` is a dict of dicts of dicts indexed by generation, than statistic name, than statistic component. For convenience, calling `sim.results` returns the most recent generation only, and is equivalent to calling `sim.results_store[sim.generation]`:

In [3]:
sim.results

{'sample_statistics': {'vcov': phenotype_name                                           height  \
  component_name                                  additiveGenetic   
  vorigin_relative                                        proband   
  phenotype_name component_name  vorigin_relative                   
  height         additiveGenetic proband                 0.564869   
  BMD            additiveGenetic proband                -0.027414   
  height         additiveNoise   proband                 0.020385   
  BMD            additiveNoise   proband                -0.009606   
  height         phenotype       proband                 0.585255   
  BMD            phenotype       proband                -0.037020   
  
  phenotype_name                                              BMD        height  \
  component_name                                  additiveGenetic additiveNoise   
  vorigin_relative                                        proband       proband   
  phenotype_name component_na

## Skipping generations

We dont always care to run each statistic each generation. We can pick and choose by altering the `statistics` attribute of a simulation object on the fly. Here we only compute sample statistics for generation zero and four:


In [4]:
sim = xft.sim.Simulation(founder_haplotypes=founder_haplotypes,
                         architecture=architecture,
                         recombination_map=recombination_map,
                         mating_regime=mating_regime,
                         statistics = [stats.SampleStatistics()])
sim.run(1)
sim.statistics = []
sim.run(3)
sim.statistics = [stats.SampleStatistics()]
sim.run(1)
xft.utils.print_tree(sim.results_store)

0: 
|__sample_statistics: 
|____means: <class 'pandas.core.series.Series'>
|____variances: <class 'pandas.core.series.Series'>
|____variance_components: <class 'pandas.core.series.Series'>
|____vcov: <class 'pandas.core.frame.DataFrame'>
|____corr: <class 'pandas.core.frame.DataFrame'>
4: 
|__sample_statistics: 
|____means: <class 'pandas.core.series.Series'>
|____variances: <class 'pandas.core.series.Series'>
|____variance_components: <class 'pandas.core.series.Series'>
|____vcov: <class 'pandas.core.frame.DataFrame'>
|____corr: <class 'pandas.core.frame.DataFrame'>


## Sample and mating statistics

Two of the helpful sets of statistics are provided by `stats.SampleStatistics` and `stats.MatingStatistics`. We've seen plenty of the former already, but the latter will produce statistics at the level of the mate pairing (rather than the offspring, for which there could be multiple for a give pairing). By default, `stats.MatingStatistics` will generate information about reproduction rates and cross-mate cross-trait phenotypic correlation matrices:


In [6]:
sim.statistics = [stats.MatingStatistics()]
sim.run(1)
sim.results

{'mating_statistics': {'n_reproducing_pairs': 1000,
  'n_total_offspring': 2000,
  'mean_n_offspring_per_pair': 2.0,
  'mean_n_female_offspring_per_pair': 0.998,
  'mate_correlations': component                height.phenotype.mother  BMD.phenotype.mother  \
  component                                                                
  height.phenotype.mother                 1.000000             -0.041244   
  BMD.phenotype.mother                   -0.041244              1.000000   
  height.phenotype.father                 0.020241              0.014931   
  BMD.phenotype.father                   -0.044721              0.031786   
  
  component                height.phenotype.father  BMD.phenotype.father  
  component                                                               
  height.phenotype.mother                 0.020241             -0.044721  
  BMD.phenotype.mother                    0.014931              0.031786  
  height.phenotype.father                 1.000000        

We can expand this to include all phenotype components (e.g., heritable and non-heritable portions) by supplying the `full` flag:

In [7]:
sim.statistics = [stats.MatingStatistics(full=True)]
sim.run(1)
sim.results['mating_statistics']['mate_correlations']

component,height.additiveGenetic.mother,BMD.additiveGenetic.mother,height.additiveNoise.mother,BMD.additiveNoise.mother,height.phenotype.mother,BMD.phenotype.mother,height.additiveGenetic.father,BMD.additiveGenetic.father,height.additiveNoise.father,BMD.additiveNoise.father,height.phenotype.father,BMD.phenotype.father
component,,,,,,,,,,,,
height.additiveGenetic.mother,1.000000,-0.033797,0.007527,-0.017540,0.736154,-0.034842,-0.016572,0.026994,-0.029024,-0.049592,-0.031165,-0.022927
BMD.additiveGenetic.mother,-0.033797,1.000000,-0.040517,0.008326,-0.052131,0.637463,-0.016763,0.010287,-0.010406,-0.010431,-0.019014,-0.001869
height.additiveNoise.mother,0.007527,-0.040517,1.000000,0.018875,0.682335,-0.011024,0.051474,0.024576,-0.000106,0.043680,0.037225,0.051169
BMD.additiveNoise.mother,-0.017540,0.008326,0.018875,1.000000,-0.000047,0.775762,0.017021,-0.047012,0.009792,-0.005590,0.018796,-0.034651
height.phenotype.mother,0.736154,-0.052131,0.682335,-0.000047,1.000000,-0.032933,0.022724,0.036368,-0.021290,-0.006691,0.002412,0.017872
BMD.phenotype.mother,-0.034842,0.637463,-0.011024,0.775762,-0.032933,1.000000,0.002536,-0.029732,0.000978,-0.010890,0.002483,-0.027878
height.additiveGenetic.father,-0.016572,-0.016763,0.051474,0.017021,0.022724,0.002536,1.000000,-0.076318,0.041126,0.023482,0.751691,-0.029847
BMD.additiveGenetic.father,0.026994,0.010287,0.024576,-0.047012,0.036368,-0.029732,-0.076318,1.000000,-0.034649,-0.065565,-0.078166,0.587447
height.additiveNoise.father,-0.029024,-0.010406,-0.000106,0.009792,-0.021290,0.000978,0.041126,-0.034649,1.000000,0.019193,0.689872,-0.006631


## Estimators

So far, `xftsim` includes routines for genome-wide association studies and Haseman Elston regression for estimating heritability and genetic correlations, which we demonstrate briefly below. For further details see the API documentation: {ref}`stats <xftsim:xftsim.stats module>`.


In [12]:
sim.statistics = [stats.GWAS_Estimator(), stats.HasemanElstonEstimator(randomized=True)]
sim.run(1)

### GWAS

The GWAS estimator returns a 3-dimensional array consisting of slopes, standard errors, test statistics and p-values:

In [14]:
sim.results['GWAS']['estimates']

<xarray.DataArray (variant: 800, statistic: 4, component: 2)>
array([[[-3.4371983e-02, -6.3749880e-02],
        [ 2.2358660e-02,  2.2326374e-02],
        [-1.5373007e+00, -2.8553619e+00],
        [ 1.8756219e+00,  1.9956568e+00]],

       [[ 1.9818721e-02, -4.3654725e-02],
        [ 2.2367459e-02,  2.2350527e-02],
        [ 8.8605154e-01, -1.9531854e+00],
        [ 3.7569639e-01,  1.9490629e+00]],

       [[-3.7906948e-02,  9.7090472e-03],
        [ 2.2355804e-02,  2.2370830e-02],
        [-1.6956199e+00,  4.3400478e-01],
        [ 1.9098864e+00,  6.6433185e-01]],

       ...,

       [[ 6.1049838e-02,  6.4005225e-04],
        [ 2.2330167e-02,  2.2371890e-02],
        [ 2.7339625e+00,  2.8609663e-02],
        [ 6.3129230e-03,  9.7717875e-01]],

       [[ 4.4225961e-02, -5.6369202e-03],
        [ 2.2349987e-02,  2.2371521e-02],
        [ 1.9787914e+00, -2.5196856e-01],
        [ 4.7976617e-02,  1.1989087e+00]],

       [[-4.6230543e-02,  4.8488475e-02],
        [ 2.2347959e-02,  2.2345563e-02],
        [-2.0686696e+00,  2.1699374e+00],
        [ 1.9612945e+00,  3.0129083e-02]]], dtype=float32)
Coordinates: (12/14)
  * component         (component) <U24 'height.phenotype.proband' 'BMD.phenot...
    phenotype_name    (component) object 'height' 'BMD'
    component_name    (component) object 'phenotype' 'phenotype'
    vorigin_relative  (component) int64 -1 -1
  * variant           (variant) <U5 '0.d' '1.d' '2.d' ... '798.d' '799.d'
    vid               (variant) object '0' '1' '2' '3' ... '797' '798' '799'
    ...                ...
    zero_allele       (variant) object 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A'
    one_allele        (variant) object 'G' 'G' 'G' 'G' 'G' ... 'G' 'G' 'G' 'G'
    af                (variant) float64 0.2201 0.1143 0.6603 ... 0.7992 0.5468
    pos_bp            (variant) float64 nan nan nan nan nan ... nan nan nan nan
    pos_cM            (variant) float64 nan nan nan nan nan ... nan nan nan nan
  * statistic         (statistic) <U4 'beta' 'se' 't' 'p'

### HE Regression
The HE regression estimator produces genetic (co)variance and correlation estimates. We highly recommend setting the `randomized` flag to `True` (as we have done above) for all but the smallest problems.

In [9]:
sim.results['HE_regression']

{'cov_HE': phenotype_name                                    height       BMD
 component_name                                 phenotype phenotype
 vorigin_relative                                 proband   proband
 phenotype_name component_name vorigin_relative                    
 height         phenotype      proband           0.438210 -0.007609
 BMD            phenotype      proband          -0.007609  0.429780,
 'corr_HE': phenotype_name                                    height       BMD
 component_name                                 phenotype phenotype
 vorigin_relative                                 proband   proband
 phenotype_name component_name vorigin_relative                    
 height         phenotype      proband           1.000000 -0.017534
 BMD            phenotype      proband          -0.017534  1.000000}

### Coming soon

We plan to implement the following additional estimation procedures in the near-term:

 - polygenic score computation
 - partitioned HE-regression
 - LD score regression
 - a general cross-validation wrapper for exmaining out-of-sample performance of all estimators 

### Creating custom estimators

:::{note}

Tutorial coming soon.

:::
